In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ffipy import FFIEC_Client
from io import StringIO

In [23]:
client = FFIEC_Client()
# get a list of RSSD codes for filers on 9/30/2010
filers_RSSD_09302010 = client.retrieve_filers_since_date(ds_name='Call',
                                   reporting_pd_end='9/30/2010',
                                   last_update_date='9/30/2010')
# check how many filers - 5908
len(filers_RSSD_09302010)

7094

In [25]:
# Testing Call report for 1 filer
ds_name = 'Call'    # Pull Call report data
end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
fiID_type = 'ID_RSSD'  # Type of financial inst identifier
fiID = 688556    # Indentifier for Wyomin Bank and Trust
fmt = 'SDF'  # Pull report as PDF
# <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
outfile = None  # Output file path
return_result = True  # If True, method returns the data
results_bytes = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                              fiID_type=fiID_type, fiID=fiID,
                              facsimile_fmt=fmt, outfile=None,
                              return_result=True)
report = StringIO(str(results_bytes, 'utf-8'))
report_df = pd.read_csv(report, sep=';')



In [27]:
report_df.head()

,Call Date,Bank RSSD Identifier,MDRM #,Value,Last Update,Short Definition,Call Schedule,Line Number
0,20100930,688556,RCON0010,14201,20101012,Cash and balances dues from depository institu...,RCR,34
1,20100930,688556,RCON0020,0,20101012,Cash items in process of collection and unpost...,RCA,1a
2,20100930,688556,RCON0071,12625,20101012,Interest-bearing balances,RC,1b
3,20100930,688556,RCON0073,0,20101012,Foreign branches of other U.S. banks,RCA,3a
4,20100930,688556,RCON0074,0,20101012,Other banks in foreign countries and foreign c...,RCA,3b


In [28]:
# Prepare dictionary of selected financials that are needed for CAMELS
selected_financials = {'RIAD3210' : 'Total equity capital', # Alt can be RCONG105
                       'RCON2170' : 'Total assets', # Ok
                       'RCON3360' : 'Total loans', #ok
                       'RCON3465' : '1-4 family residential loans', 
                       'RCON3466' : 'Other real estate loans',
                       'RCON3387' : 'Commercial and industrial loans',
                       'RCONB561' : 'Credit cards',
                       'RCON3123' : 'Allowance for loan losses', # OK
                       'RIAD4093' : 'Total noninterest expense', #OK
                       'RIAD4340' : 'Net Income attributable', # ok
                       'RCON2215' : 'Total transaction deposits', # ok
                       'RCON2385' : 'Total nontransaction deposits', # ok
                       'RCON1773' : 'Available-for-sale Fair Value'
                      } 


In [29]:
# Expected 13 columns, store column names in the list columns
len(selected_financials.keys())
columns = list(selected_financials.keys())
len(columns)

13

In [ ]:
# Pull 7000 reports for filers_RSSD_09302010, drop all values but 13 selected metrics -- returns only 27
all_data_df = pd.DataFrame()
fin_data_dic = {}
for i in range(len(filers_RSSD_09302010)):

    ds_name = 'Call'    # Pull Call report data
    end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
    fiID_type = 'ID_RSSD'  # Type of financial inst identifier (ID_RSSD, CERT )
    fiID = filers_RSSD_09302010[i]   # Indentifier for Wyomin Bank and Trust
    print("counter at ", i, " pulling RSSD# ", fiID)
    fmt = 'SDF'  # Pull report as PDF
    # <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
    #outfile = 'test8.PDF'  # Output file path
    return_result = True  # If True, method returns the data
    try:
        f = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                                      fiID_type=fiID_type, fiID=fiID,
                                      facsimile_fmt=fmt, outfile=False,
                                      return_result=True)
        report = StringIO(str(f, 'utf-8'))
        report_df = pd.read_csv(report, sep=';')
        report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       'Short Definition', 'Call Schedule', 'Line Number']
        values = list(report_df[report_df.ValueID.isin(columns)].Value)
        fin_data_dic[fiID] = values
        all_data_df = all_data_df.append(report_df, ignore_index=True)
    except:
        next 

In [ ]:
# Inspect the data pulled
healthy_after_2010Q3 = pd.DataFrame.from_dict(fin_data_dic, orient='index')
healthy_after_2010Q3.columns = columns
values = list(selected_financials.values())
healthy_after_2010Q3.columns = values
healthy_after_2010Q3


#store the data
#healthy_after_2010Q3.to_csv('data/healthy_after_2010Q3')